In [4]:
# Login the website

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
import time
import pandas as pd
from datetime import datetime

In [6]:
website = 'https://app.chainalysis.com/login?redirect=reactor'
cService = webdriver.ChromeService(executable_path=r'D:\Desktop\Ah Tim\Web Scraping\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service = cService)
driver.get(website)
driver.maximize_window()
your_username = 'check@okx.com'
your_password = 'trkkcyngdu'
username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'MuiFilledInput-input'))
    )
username_input.send_keys(your_username)
continue_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'MuiButton-containedPrimary'))
    )
continue_button.click()
password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'password'))
    )
password_input.send_keys(your_password)
continue_button = driver.find_element(By.XPATH, '//button[@type="submit" and contains(text(), "Continue")]')
continue_button.click()

### ENTER INTO THE SEARCH PHASE

In [7]:
graphs_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Graphs")]'))
    )

# Click on the "Graphs" element
graphs_element.click()

In [8]:
new_graph_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "New graph")]/ancestor::a'))
    )

# Click on the "Graphs" element
new_graph_link.click()

In [9]:
max_iterations = 12010  # Set the maximum number of iterations
current_iteration = 0
scroll_increment = 3
processed_elements = set()
max_retries = 3  # Set the maximum number of retries for each action
element_index = 0  # Initialize the element index outside the loop
max_elements_to_click = 3  # Number of elements to click before scrolling
sleep_after_scroll = 2  # Time to sleep after scrolling (adjust as needed)

In [10]:
while current_iteration < max_iterations:
    try:
        try:
            # Dismiss overlay
            overlay_locator = (By.CLASS_NAME, 'cdk-overlay-backdrop')
            overlay = WebDriverWait(driver, 10).until(EC.invisibility_of_element_located(overlay_locator))

        except StaleElementReferenceException:
            print("Stale Element Reference: Retrying overlay dismissal...")
            continue

        # Attempt to click the "eye" element
        eye_clicked = False
        for retry_eye_click in range(max_retries):
            try:
                eye_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]')
                eye = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(eye_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(eye).perform()

                # Click on the SVG element
                ActionChains(driver).click(eye).perform()

                eye_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on eye element: {e}. Retrying...")

        if not eye_clicked:
            print("Failed to click on the eye. Skipping iteration.")
            continue

        # Attempt to click the "scam" element
        scam_clicked = False
        for retry_scam_click in range(max_retries):
            try:
                scam_locator = (By.XPATH, '//p[contains(text(), "Scam")]')
                scam_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(scam_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(scam_element).perform()

                # Click on the scam element
                ActionChains(driver).click(scam_element).perform()

                scam_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on scam element: {e}. Retrying...")

        if not scam_clicked:
            print("Failed to click on the scam. Skipping iteration.")
            continue

        # Clicking the 'cdk-virtual-scroll-content-wrapper' container
        WebDriverWait(driver, 20).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        parent_container_locator = (By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
        parent_container = WebDriverWait(driver, 20).until(EC.presence_of_element_located(parent_container_locator))

        parent_container.click()

        # Continuing with the loop and scraping logic
        processed_elements = set()

        # Find all elements within the parent container
        elements = parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span')

        # Loop through each element starting from the saved index
        for idx in range(element_index, len(elements)):
            element = elements[idx]

            # Check if the element has already been processed
            if element.text in processed_elements:
                continue

            for retry in range(max_retries):
                try:
                    # Refresh the parent container before interacting with the element
                    parent_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located(parent_container_locator))

                    # Wait for the element to be clickable
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(element).perform()

                    # Click the element
                    element.click()

                    # Add the element to the set of processed elements
                    processed_elements.add(element.text)

                    # Perform other actions as needed

                    xbutton_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.csui-icon-button'))
                    )

                    # Click on the SVG element
                    xbutton_element.click()

                    # Break out of the retry loop if successful
                    break

                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry {retry + 1}")

            # Increment the iteration counter
            current_iteration += 1

            # Scroll down with retries
            for retry_scroll in range(max_retries):
                try:
                    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)
                    break
                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry scrolling {retry_scroll + 1}")

            # Clicking the close icon
            close_icon_clicked = False
            for retry_close_icon_click in range(max_retries):
                try:
                    close_icon_locator = (By.CSS_SELECTOR, 'svg[data-testid="CloseOutlinedIcon"]')
                    close_icon = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(close_icon_locator))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(close_icon).perform()

                    # Click on the close icon
                    ActionChains(driver).click(close_icon).perform()

                    close_icon_clicked = True
                    break

                except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                    print(f"Error clicking on close icon: {e}. Retrying...")

            if not close_icon_clicked:
                print("Failed to click on the close icon.")

            # Update the element index to start from the next element in the next iteration
            element_index = idx + 1

            # Wait for a short moment to let the content load
            time.sleep(1)

    except Exception as e:
        # Handle other exceptions if needed
        print(f"Error: {e}")


Stale Element Reference: Retry scrolling 1
Stale Element Reference: Retry scrolling 2
Stale Element Reference: Retry scrolling 3
Error: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61BE85E42+3538674]
	(No symbol) [0x00007FF61BAA4C02]
	(No symbol) [0x00007FF61B955AEB]
	(No symbol) [0x00007FF61B95AD39]
	(No symbol) [0x00007FF61B95D150]
	(No symbol) [0x00007FF61B95D210]
	(No symbol) [0x00007FF61B99723B]
	(No symbol) [0x00007FF61B9BF0AA]
	(No symbol) [0x00007FF61B99124A]
	(No symbol) [0x00007FF61B9BF2C0]
	(No symbol) [0x00007FF61B9DBDE3]
	(No symbol) [0x00007FF61B9BEE53]
	(No symbol) [0x00007FF61B98F514]
	(No symbol) [0x00007FF61B990631]
	GetHandleVerifier [0x00007FF61BEB6CAD+3738973]
	GetHandleVerifier [0x00007FF61BF0C506+4089270]
	GetHandleVeri

KeyboardInterrupt: 

In [ ]:
element_index = 0  # Initialize the element index outside the loop

while current_iteration < max_iterations:
    try:
        try:
            # Dismiss overlay
            overlay_locator = (By.CLASS_NAME, 'cdk-overlay-backdrop')
            overlay = WebDriverWait(driver, 10).until(EC.invisibility_of_element_located(overlay_locator))

        except StaleElementReferenceException:
            print("Stale Element Reference: Retrying overlay dismissal...")
            continue

        # Attempt to click the "eye" element
        eye_clicked = False
        for retry_eye_click in range(max_retries):
            try:
                eye_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]')
                eye = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(eye_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(eye).perform()

                # Click on the SVG element
                ActionChains(driver).click(eye).perform()

                eye_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on eye element: {e}. Retrying...")

        if not eye_clicked:
            print("Failed to click on the eye. Skipping iteration.")
            continue

        # Attempt to click the "scam" element
        scam_clicked = False
        for retry_scam_click in range(max_retries):
            try:
                scam_locator = (By.XPATH, '//p[contains(text(), "Scam")]')
                scam_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(scam_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(scam_element).perform()

                # Click on the scam element
                ActionChains(driver).click(scam_element).perform()

                scam_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on scam element: {e}. Retrying...")

        if not scam_clicked:
            print("Failed to click on the scam. Skipping iteration.")
            continue

        # Clicking the 'cdk-virtual-scroll-content-wrapper' container
        WebDriverWait(driver, 20).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        parent_container_locator = (By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
        parent_container = WebDriverWait(driver, 20).until(EC.presence_of_element_located(parent_container_locator))

        # Get the initial number of elements
        initial_element_count = len(parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span'))

        # Perform scrolling to load more elements
        while True:
            driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)
            time.sleep(1)  # Adjust the sleep time based on the page load speed

            # Get the current number of elements after scrolling
            current_element_count = len(parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span'))

            # Break the loop if no new elements are loaded
            if current_element_count == initial_element_count:
                break

            # Update the initial element count
            initial_element_count = current_element_count

        # Continuing with the loop and scraping logic
        processed_elements = set()

        # Find all elements within the parent container
        elements = parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span')

        # Loop through each element starting from the saved index
        for idx in range(element_index, len(elements)):
            element = elements[idx]

            # Check if the element has already been processed
            if element.text in processed_elements:
                continue

            for retry in range(max_retries):
                try:
                    # Refresh the parent container before interacting with the element
                    parent_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located(parent_container_locator))

                    # Wait for the element to be clickable
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(element).perform()

                    # Click the element
                    element.click()

                    # Add the element to the set of processed elements
                    processed_elements.add(element.text)

                    # Perform other actions as needed

                    xbutton_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.csui-icon-button'))
                    )

                    # Click on the SVG element
                    xbutton_element.click()

                    # Break out of the retry loop if successful
                    break

                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry {retry + 1}")

            # Increment the iteration counter
            current_iteration += 1

            # Scroll down with retries
            for retry_scroll in range(max_retries):
                try:
                    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)
                    break
                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry scrolling {retry_scroll + 1}")

             # Clicking the close icon
            close_icon_clicked = False
            for retry_close_icon_click in range(max_retries):
                try:
                    close_icon_locator = (By.CSS_SELECTOR, 'svg[data-testid="CloseOutlinedIcon"]')
                    close_icon = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(close_icon_locator))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(close_icon).perform()

                    # Click on the close icon
                    ActionChains(driver).click(close_icon).perform()

                    close_icon_clicked = True
                    break

                except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                    print(f"Error clicking on close icon: {e}. Retrying...")

            if not close_icon_clicked:
                print("Failed to click on the close icon.")

            # Update the element index to start from the next element in the next iteration
            element_index = idx + 1

            # Wait for a short moment to let the content load
            time.sleep(1)

    except Exception as e:
        # Handle other exceptions if needed
        print(f"Error: {e}")


In [ ]:
element_index = 0  # Initialize the element index outside the loop

while current_iteration < max_iterations:
    try:
        try:
            # Dismiss overlay
            overlay_locator = (By.CLASS_NAME, 'cdk-overlay-backdrop')
            overlay = WebDriverWait(driver, 10).until(EC.invisibility_of_element_located(overlay_locator))

        except StaleElementReferenceException:
            print("Stale Element Reference: Retrying overlay dismissal...")
            continue

        # Attempt to click the "eye" element
        eye_clicked = False
        for retry_eye_click in range(max_retries):
            try:
                eye_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]')
                eye = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(eye_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(eye).perform()

                # Click on the SVG element
                ActionChains(driver).click(eye).perform()

                eye_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on eye element: {e}. Retrying...")

        if not eye_clicked:
            print("Failed to click on the eye. Skipping iteration.")
            continue

        # Attempt to click the "scam" element
        scam_clicked = False
        for retry_scam_click in range(max_retries):
            try:
                scam_locator = (By.XPATH, '//p[contains(text(), "Scam")]')
                scam_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(scam_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(scam_element).perform()

                # Click on the scam element
                ActionChains(driver).click(scam_element).perform()

                scam_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on scam element: {e}. Retrying...")

        if not scam_clicked:
            print("Failed to click on the scam. Skipping iteration.")
            continue

        # Clicking the 'cdk-virtual-scroll-content-wrapper' container
        WebDriverWait(driver, 20).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        parent_container_locator = (By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
        parent_container = WebDriverWait(driver, 20).until(EC.presence_of_element_located(parent_container_locator))

        parent_container.click()

        # Continuing with the loop and scraping logic
        processed_elements = set()

        # Find all elements within the parent container
        elements = parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span')

        # Loop through each element starting from the saved index
        for idx in range(element_index, len(elements)):
            element = elements[idx]

            # Check if the element has already been processed
            if element.text in processed_elements:
                continue

            for retry in range(max_retries):
                try:
                    # Refresh the parent container before interacting with the element
                    parent_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located(parent_container_locator))

                    # Wait for the element to be clickable
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(element).perform()

                    # Click the element
                    element.click()

                    # Add the element to the set of processed elements
                    processed_elements.add(element.text)

                    # Perform other actions as needed

                    xbutton_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.csui-icon-button'))
                    )

                    # Click on the SVG element
                    xbutton_element.click()

                    # Break out of the retry loop if successful
                    break

                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry {retry + 1}")

            # Increment the iteration counter
            current_iteration += 1

            # Scroll down with retries
            for retry_scroll in range(max_retries):
                try:
                    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)
                    break
                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry scrolling {retry_scroll + 1}")

            # Clicking the close icon
            close_icon_clicked = False
            for retry_close_icon_click in range(max_retries):
                try:
                    close_icon_locator = (By.CSS_SELECTOR, 'svg[data-testid="CloseOutlinedIcon"]')
                    close_icon = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(close_icon_locator))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(close_icon).perform()

                    # Click on the close icon
                    ActionChains(driver).click(close_icon).perform()

                    close_icon_clicked = True
                    break

                except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                    print(f"Error clicking on close icon: {e}. Retrying...")

            if not close_icon_clicked:
                print("Failed to click on the close icon.")

            # Update the element index to start from the next element in the next iteration
            element_index = idx + 1

            # Wait for a short moment to let the content load
            time.sleep(1)

    except Exception as e:
        # Handle other exceptions if needed
        print(f"Error: {e}")


In [ ]:
#### DONE!!!


element_index = 0  # Initialize the element index outside the loop

while current_iteration < max_iterations:
    try:
        try:
        # Dismiss overlay
            overlay_locator = (By.CLASS_NAME, 'cdk-overlay-backdrop')
            overlay = WebDriverWait(driver, 10).until(EC.invisibility_of_element_located(overlay_locator))

        except StaleElementReferenceException:
            print("Stale Element Reference: Retrying overlay dismissal...")
            continue

        # Attempt to click the "eye" element
        eye_clicked = False
        for retry_eye_click in range(max_retries):
            try:
                eye_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]')
                eye = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(eye_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(eye).perform()

                # Click on the SVG element
                ActionChains(driver).click(eye).perform()

                eye_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on eye element: {e}. Retrying...")

        if not eye_clicked:
            print("Failed to click on the eye. Skipping iteration.")
            continue

        # Attempt to click the "scam" element
        scam_clicked = False
        for retry_scam_click in range(max_retries):
            try:
                scam_locator = (By.XPATH, '//p[contains(text(), "Scam")]')
                scam_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(scam_locator))

                # Move to the element before clicking
                ActionChains(driver).move_to_element(scam_element).perform()

                # Click on the scam element
                ActionChains(driver).click(scam_element).perform()

                scam_clicked = True
                break

            except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                print(f"Error clicking on scam element: {e}. Retrying...")

        if not scam_clicked:
            print("Failed to click on the scam. Skipping iteration.")
            continue

        # Clicking the 'cdk-virtual-scroll-content-wrapper' container
        WebDriverWait(driver, 20).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        parent_container_locator = (By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
        parent_container = WebDriverWait(driver, 20).until(EC.presence_of_element_located(parent_container_locator))

        parent_container.click()

        # Continuing with the loop and scraping logic
        processed_elements = set()

        # Find all elements within the parent container
        elements = parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span')

        # Loop through each element starting from the saved index
        for idx in range(element_index, len(elements)):
            element = elements[idx]

            # Check if the element has already been processed
            if element.text in processed_elements:
                continue

            for retry in range(max_retries):
                try:
                    # Wait for the element to be clickable
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(element).perform()

                    # Click the element
                    element.click()

                    # Add the element to the set of processed elements
                    processed_elements.add(element.text)

                    # Perform other actions as needed

                    xbutton_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.csui-icon-button'))
                    )

                    # Click on the SVG element
                    xbutton_element.click()

                    # Break out of the retry loop if successful
                    break

                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry {retry + 1}")

            # Increment the iteration counter
            current_iteration += 1

            # Scroll down with retries
            for retry_scroll in range(max_retries):
                try:
                    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)
                    break
                except StaleElementReferenceException:
                    print(f"Stale Element Reference: Retry scrolling {retry_scroll + 1}")

            # Clicking the close icon
            close_icon_clicked = False
            for retry_close_icon_click in range(max_retries):
                try:
                    close_icon_locator = (By.CSS_SELECTOR, 'svg[data-testid="CloseOutlinedIcon"]')
                    close_icon = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(close_icon_locator))

                    # Move to the element before clicking
                    ActionChains(driver).move_to_element(close_icon).perform()

                    # Click on the close icon
                    ActionChains(driver).click(close_icon).perform()

                    close_icon_clicked = True
                    break

                except (ElementClickInterceptedException, StaleElementReferenceException) as e:
                    print(f"Error clicking on close icon: {e}. Retrying...")

            if not close_icon_clicked:
                print("Failed to click on the close icon.")

            # Update the element index to start from the next element in the next iteration
            element_index = idx + 1

            # Wait for a short moment to let the content load
            time.sleep(1)

    except Exception as e:
        # Handle other exceptions if needed
        print(f"Error: {e}")

# Start Web scraping

In [ ]:
# its u
# Assuming 'driver' is your WebDriver instance
parent_container = driver.find_element(By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
max_iterations = 12010  # Set the maximum number of iterations
current_iteration = 0
scroll_increment = 3
processed_elements = set()

while current_iteration < max_iterations:
    try:
        # Locate and click the element
        eye = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]'))
        )
        eye.click()

        # Add the element to the set of processed elements
        processed_elements.add(eye)

        # Perform other actions as needed

    except StaleElementReferenceException:
        print("Stale Element Reference: Retrying...")
        continue

    except Exception as e:
        print(f"Error interacting with element: {e}")

    # Increment the iteration counter
    current_iteration += 1

    # Scroll down
    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)

    # Wait for a short moment to let the content load
    time.sleep(1)

    try:
        # Locate and click the "Scam" element
        scam_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "Scam")]'))
        )
        scam_element.click()

        # Add the element to the set of processed elements
        processed_elements.add(scam_element)

        # Perform other actions as needed

    except StaleElementReferenceException:
        print("Stale Element Reference: Retrying...")
        continue

    except Exception as e:
        print(f"Error interacting with 'Scam' element: {e}")
        
        # Find all elements within the parent container
        try:
            element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper'))
            )
    # Continue with your code after the element is located
        except Exception as e:
            print(f"Error: {e}")
            
        # Loop through each element
        for element in elements:
            # Check if the element has already been processed
            if element in processed_elements:
                continue

            try:
                # Wait for the element to be clickable
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                # Click the element
                element.click()
                
                xbutton_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.csui-icon-button'))
                )

# Click on the SVG element
                xbutton_element.click()

                # Add the element to the set of processed elements
                processed_elements.add(element)
                # Wait for the h2 element with data-testid 'entity-title' to be present
                h2_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'h2[data-testid="entity-title"]'))
                    )

                # Extract the text from the h2 element
                cluster = h2_element.text
                print(cluster)

                h2_element.click()

                span_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.MuiTypography-inputText'))
                    )

                    # Extract the text from the span element
                chainalysis_name = span_element.text
                print(chainalysis_name)

                span_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.tooltip-content-placeholder span span'))
                    )

                    # Extract the text from the span element
                root_address = span_element.text
                print(root_address)

                xbutton_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[3]/div/form/h3/button'))
                    )
                xbutton_element.click()

                # Assuming 'driver' is your WebDriver instance
                elements = driver.find_elements(By.CSS_SELECTOR, '.grid-y.text-right .cell.mono.ellipsis')

                # Define keys for each value
                keys = ['Balance', 'Sent', 'Received', 'Total fees', 'Transfers', 'Withdrawals', 'Deposits', 'Addresses']

                # Create a dictionary with keys and corresponding values
                data_dict = dict(zip(keys, [element.text for element in elements]))

                print("Extracted data dictionary:", data_dict)

                transfers_link = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "/graph/new/cluster/asset-v2:bip122:000000000019d6689c085ae165831e93:native/1JujJi8h3AsPJpEnQX5pew2g5VsEHAqMVe/transfers")]'))
                )

                # Click on the "Transfers" link
                transfers_link.click()

                # Find the element
                element = driver.find_element(By.CLASS_NAME, 'icon-collapse')

                # Perform the click action
                ActionChains(driver).click(element).perform()

                # Wait for the element to be clickable
                chevron_element = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, 'icon-chevron-right'))
                    )

                # Click on the chevron element
                chevron_element.click()

                time.sleep(3)  # Adjust the sleep duration based on your page load time
                time_element = driver.find_element(By.CSS_SELECTOR, 'div.shrink span.mono')
                time_text = time_element.get_attribute("innerText")
                print("Time Text:", time_text)

                int(data_dict.get('Addresses', 0)) > 10
                output_list = [data_dict['Addresses']]
                print(output_list)

                # Check the condition for 'Addresses'
                if int(data_dict.get('Addresses', 0)) > 10:
                    output_list = [data_dict['Addresses']]
                    print(output_list)
                else:
                    address_link_locator = (By.XPATH, '//a[contains(@href, "/addresses")]')
                    address_link_element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(address_link_locator))

                    address_link_element.click()
                    addresses_locator = (By.CSS_SELECTOR, 'span.copy-text.ellipsis.ng-star-inserted')
                    addresses_elements = WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located(addresses_locator))
                    addresses_content_list = [element.text.strip() for element in addresses_elements]
                    print("Addresses Content List:", addresses_content_list)

                from selenium.webdriver.common.by import By
                from selenium.webdriver.support.ui import WebDriverWait
                from selenium.webdriver.support import expected_conditions as EC
                from selenium.common.exceptions import TimeoutException

                # Replace 'your_locator_here' with the appropriate locator for the OSINT link
                osint_link_locator = (By.CSS_SELECTOR, 'a[href*="/osint"]')

                try:
                    # Wait for the OSINT link to be present on the page
                    osint_link = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(osint_link_locator)
                    )

                    # If the link exists, click on it or perform any other actions you want
                    osint_link.click()

                    # Now you can scrape the data related to OSINT
                    # Replace 'your_locator_here' with the appropriate locator for the SVG element
                    svg_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root.css-1onxyik[data-testid="FilterAltOutlinedIcon"]')

                # Wait for the SVG element to be clickable with an extended waiting time (e.g., 20 seconds)
                    svg_element = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(svg_locator))

                # Click on the SVG element
                    svg_element.click()

                # Replace 'your_locator_here' with the appropriate locator for the elements
                    elements_locator = (By.CSS_SELECTOR, '.MuiTypography-root.MuiTypography-body2.MuiFormControlLabel-label.css-me8ks0')

                # Wait for the elements to be present on the page
                    elements = WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located(elements_locator))

                # Create a list of texts excluding "Select all"
                    text_list = [element.text for element in elements if element.text != "Select all"]

                # Print the resulting list
                    print(text_list)


                except TimeoutException:
                    # Handle the case when OSINT link is not found
                    pass

                # Simulate clicking anywhere on the page to close the dropdown
                action_chains = ActionChains(driver)
                action_chains.move_by_offset(1, 1).click().perform()

                # Replace 'your_locator_here' with the appropriate locator for the element you want to capture
                element_locator = (By.XPATH, '/html/body/cs-ngx-bootstrap/cs-ngx-app-root/div/div/div/div/div/cs-ngx-main/div/div[2]/div/div[1]/cs-ngx-coin-cluster-info-panel')

                # Wait for the element to be present on the page
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(element_locator)
                )

                # Capture a screenshot of the element
                element.screenshot('element_screenshot.png')
                from PIL import Image

                driver.save_screenshot("element_screenshot.png")
                img = Image.open("element_screenshot.png")
                img.show()

                cluster_ = []
                chainalysis_name_ = []
                root_address_ = []
                balance_ = []
                sent_ = []
                received_ = []
                transfers_ = []
                withdrawals_ = []
                deposits_ = []
                addresses_ = []
                transfer_last_time_ = []
                address_list_ = []
                screenshot_ = []

                cluster_.append(cluster)
                chainalysis_name_.append(chainalysis_name)
                root_address_.append(root_address)
                addresses.append(output_list)

                print(cluster_)
                print(chainalysis_name_)
                print(root_address_)
                print(addresses_content_list)
                print(text_list)

                from datetime import datetime

                # Convert the string to a datetime object
                time_object = datetime.strptime(time_text, "%m/%d/%Y %I:%M %p")

                # Format the datetime object as a string
                formatted_time = time_object.strftime("%m/%d/%Y %I:%M %p")

                # Append the formatted string to a list
                time_list = [formatted_time]

                # Print the result
                print("Transfer Last time:", time_list)

                import re

                def extract_numeric_value(data_dict, key):
                    # Initialize an empty list to store the extracted values
                    result_list = []

                    # Extract numeric value from the specified key using regular expression
                    match = re.search(r'(\d+\.\d+|\d+)', data_dict.get(key, ''))

                    # Check if a match is found
                    if match:
                        # Append the extracted value to the result_list
                        result_list.append(match.group(1))

                    return result_list

                # Specify the keys for which you want to extract numeric values
                keys_to_extract = ['Balance', 'Sent', 'Received', 'Transfers', 'Withdrawals', 'Deposits', 'Addresses']

                # Initialize empty lists to store the extracted values
                balance_list, sent_list, received_list, transfers_list, withdrawals_list, deposits_list, addresses_list = (
                    extract_numeric_value(data_dict, key) for key in keys_to_extract
                )

                # Print the results
                print("Balance List:", balance_list)
                print("Sent List:", sent_list)
                print("Received List:", received_list)
                print("Transfers List:", transfers_list)
                print("Withdrawals List:", withdrawals_list)
                print("Deposits List:", deposits_list)
                print("Addresses List:", addresses_list)

                # Join elements of text_list with a comma
                osint_source_text = ', '.join(text_list)

                def join_addresses_content(addresses_content_list):
                    # Check if the length is greater than one
                    if len(addresses_content_list) > 1:
                        # Join the elements with a comma and return
                        return ', '.join(addresses_content_list)
                    else:
                        # If the length is one or zero, return the element as is
                        return addresses_content_list[0] if addresses_content_list else None

                # Example usage:
                addresses_content_list = join_addresses_content(addresses_content_list)

                # Create DataFrame
                df = pd.DataFrame({
                    'Clusters列表': cluster_,
                    'Chainalysis Name': chainalysis_name_,
                    'Root address': root_address_,
                    'Balance': balance_list,
                    'Sent': sent_list,
                    'Received': received_list,
                    'Transfer': transfers_list,
                    'Withdrawals': withdrawals_list,
                    'Deposits': deposits_list,
                    'Addresses': addresses_list,
                    'Transfer Last Time': time_list,
                    'Addresses List': addresses_content_list,
                    'OSINT source': osint_source_text,
                })

                df

                df.to_excel(r'D:\Desktop\Web Scraping\output.xlsx', index=False)
                # Perform other actions as needed

            except Exception as e:
                print(f"Error interacting with element: {e}")

            # Increment the iteration counter
            current_iteration += 1

    except Exception as e:
        print(f"Error during iteration: {e}")

    # Scroll down
    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)

    # Wait for a short moment to let the content load
    time.sleep(1)


In [ ]:
# Wait for the h2 element with data-testid 'entity-title' to be present
h2_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'h2[data-testid="entity-title"]'))
    )

# Extract the text from the h2 element
cluster = h2_element.text
print(cluster)

h2_element.click()

span_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.MuiTypography-inputText'))
    )

    # Extract the text from the span element
chainalysis_name = span_element.text
print(chainalysis_name)

span_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.tooltip-content-placeholder span span'))
    )

    # Extract the text from the span element
root_address = span_element.text
print(root_address)

xbutton_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[3]/div/form/h3/button'))
    )
xbutton_element.click()

# Assuming 'driver' is your WebDriver instance
elements = driver.find_elements(By.CSS_SELECTOR, '.grid-y.text-right .cell.mono.ellipsis')

# Define keys for each value
keys = ['Balance', 'Sent', 'Received', 'Total fees', 'Transfers', 'Withdrawals', 'Deposits', 'Addresses']

# Create a dictionary with keys and corresponding values
data_dict = dict(zip(keys, [element.text for element in elements]))

print("Extracted data dictionary:", data_dict)

transfers_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "/graph/new/cluster/asset-v2:bip122:000000000019d6689c085ae165831e93:native/1JujJi8h3AsPJpEnQX5pew2g5VsEHAqMVe/transfers")]'))
)

# Click on the "Transfers" link
transfers_link.click()

# Find the element
element = driver.find_element(By.CLASS_NAME, 'icon-collapse')

# Perform the click action
ActionChains(driver).click(element).perform()

# Wait for the element to be clickable
chevron_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'icon-chevron-right'))
    )

# Click on the chevron element
chevron_element.click()

time.sleep(3)  # Adjust the sleep duration based on your page load time
time_element = driver.find_element(By.CSS_SELECTOR, 'div.shrink span.mono')
time_text = time_element.get_attribute("innerText")
print("Time Text:", time_text)

int(data_dict.get('Addresses', 0)) > 10
output_list = [data_dict['Addresses']]
print(output_list)

# Check the condition for 'Addresses'
if int(data_dict.get('Addresses', 0)) > 10:
    addresses_content_list = [data_dict['Addresses']]
    print(addresses_content_list)
else:
    address_link_locator = (By.XPATH, '//a[contains(@href, "/addresses")]')
    address_link_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(address_link_locator))
    
    address_link_element.click()
    addresses_locator = (By.CSS_SELECTOR, 'span.copy-text.ellipsis.ng-star-inserted')
    addresses_elements = WebDriverWait(driver, 10).until(
    EC.visibility_of_all_elements_located(addresses_locator))
    addresses_content_list = [element.text.strip() for element in addresses_elements]
    print("Addresses Content List:", addresses_content_list)
    
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Replace 'your_locator_here' with the appropriate locator for the OSINT link
osint_link_locator = (By.CSS_SELECTOR, 'a[href*="/osint"]')

try:
    # Wait for the OSINT link to be present on the page
    osint_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(osint_link_locator)
    )

    # If the link exists, click on it or perform any other actions you want
    osint_link.click()

    # Now you can scrape the data related to OSINT
    # Replace 'your_locator_here' with the appropriate locator for the SVG element
    svg_locator = (By.CSS_SELECTOR, 'svg.MuiSvgIcon-root.css-1onxyik[data-testid="FilterAltOutlinedIcon"]')

# Wait for the SVG element to be clickable with an extended waiting time (e.g., 20 seconds)
    svg_element = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(svg_locator))

# Click on the SVG element
    svg_element.click()

# Replace 'your_locator_here' with the appropriate locator for the elements
    elements_locator = (By.CSS_SELECTOR, '.MuiTypography-root.MuiTypography-body2.MuiFormControlLabel-label.css-me8ks0')

# Wait for the elements to be present on the page
    elements = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located(elements_locator))

# Create a list of texts excluding "Select all"
    text_list = [element.text for element in elements if element.text != "Select all"]

# Print the resulting list
    print(text_list)


except TimeoutException:
    # Handle the case when OSINT link is not found
    pass

# Simulate clicking anywhere on the page to close the dropdown
action_chains = ActionChains(driver)
action_chains.move_by_offset(1, 1).click().perform()

# Replace 'your_locator_here' with the appropriate locator for the element you want to capture
element_locator = (By.XPATH, '/html/body/cs-ngx-bootstrap/cs-ngx-app-root/div/div/div/div/div/cs-ngx-main/div/div[2]/div/div[1]/cs-ngx-coin-cluster-info-panel')

# Wait for the element to be present on the page
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(element_locator)
)

# Capture a screenshot of the element
element.screenshot('element_screenshot.png')
from PIL import Image

driver.save_screenshot("element_screenshot.png")
img = Image.open("element_screenshot.png")
img.show()

cluster_ = []
chainalysis_name_ = []
root_address_ = []
balance_ = []
sent_ = []
received_ = []
transfers_ = []
withdrawals_ = []
deposits_ = []
addresses_ = []
transfer_last_time_ = []
address_list_ = []
screenshot_ = []

cluster_.append(cluster)
chainalysis_name_.append(chainalysis_name)
root_address_.append(root_address)
addresses.append(output_list)

print(cluster_)
print(chainalysis_name_)
print(root_address_)
print(addresses_content_list)
print(text_list)

from datetime import datetime

# Convert the string to a datetime object
time_object = datetime.strptime(time_text, "%m/%d/%Y %I:%M %p")

# Format the datetime object as a string
formatted_time = time_object.strftime("%m/%d/%Y %I:%M %p")

# Append the formatted string to a list
time_list = [formatted_time]

# Print the result
print("Transfer Last time:", time_list)

import re

def extract_numeric_value(data_dict, key):
    # Initialize an empty list to store the extracted values
    result_list = []

    # Extract numeric value from the specified key using regular expression
    match = re.search(r'(\d+\.\d+|\d+)', data_dict.get(key, ''))

    # Check if a match is found
    if match:
        # Append the extracted value to the result_list
        result_list.append(match.group(1))

    return result_list

# Specify the keys for which you want to extract numeric values
keys_to_extract = ['Balance', 'Sent', 'Received', 'Transfers', 'Withdrawals', 'Deposits', 'Addresses']

# Initialize empty lists to store the extracted values
balance_list, sent_list, received_list, transfers_list, withdrawals_list, deposits_list, addresses_list = (
    extract_numeric_value(data_dict, key) for key in keys_to_extract
)

# Print the results
print("Balance List:", balance_list)
print("Sent List:", sent_list)
print("Received List:", received_list)
print("Transfers List:", transfers_list)
print("Withdrawals List:", withdrawals_list)
print("Deposits List:", deposits_list)
print("Addresses List:", addresses_list)

# Join elements of text_list with a comma
osint_source_text = ', '.join(text_list)

def join_addresses_content(addresses_content_list):
    # Check if the length is greater than one
    if len(addresses_content_list) > 1:
        # Join the elements with a comma and return
        return ', '.join(addresses_content_list)
    else:
        # If the length is one or zero, return the element as is
        return addresses_content_list[0] if addresses_content_list else None

# Example usage:
addresses_content_list = join_addresses_content(addresses_content_list)

# Create DataFrame
df = pd.DataFrame({
    'Clusters列表': cluster_,
    'Chainalysis Name': chainalysis_name_,
    'Root address': root_address_,
    'Balance': balance_list,
    'Sent': sent_list,
    'Received': received_list,
    'Transfer': transfers_list,
    'Withdrawals': withdrawals_list,
    'Deposits': deposits_list,
    'Addresses': addresses_list,
    'Transfer Last Time': time_list,
    'Addresses List': addresses_content_list,
    'OSINT source': osint_source_text,
})

df

df.to_excel(r'D:\Desktop\Web Scraping\output.xlsx', index=False)

In [ ]:
#Test Clicking the 14th cluster

button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="entity-close-button"]')

# Perform the click action
button.click()

In [11]:
eye = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg.MuiSvgIcon-root[data-testid="CsClusterOutlinedIcon"]'))
    )

# Click on the SVG element
eye.click()

In [12]:
scam_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "Scam")]'))
    )

# Click on the "Scam" element
scam_element.click()

In [13]:
checkboxes = driver.find_elements(By.CSS_SELECTOR, 'span.tooltip-content-placeholder csui-checkbox')

checkbox_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'csui-checkbox.filtered-cluster-checkbox'))
    )

checkbox_element.click()

In [14]:
# its u
# Assuming 'driver' is your WebDriver instance
parent_container = driver.find_element(By.CLASS_NAME, 'cdk-virtual-scroll-content-wrapper')
max_iterations = 12010  # Set the maximum number of iterations
current_iteration = 0
scroll_increment = 3
processed_elements = set()

while current_iteration < max_iterations:
    try:
        # Find all elements within the parent container
        elements = parent_container.find_elements(By.CSS_SELECTOR, 'cs-ngx-cluster-label span')

        # Loop through each element
        for element in elements:
            # Check if the element has already been processed
            if element in processed_elements:
                continue

            try:
                # Wait for the element to be clickable
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(element))

                # Click the element
                element.click()

                # Add the element to the set of processed elements
                processed_elements.add(element)

                # Perform other actions as needed

            except Exception as e:
                print(f"Error interacting with element: {e}")

            # Increment the iteration counter
            current_iteration += 1

    except Exception as e:
        print(f"Error during iteration: {e}")

    # Scroll down
    driver.execute_script(f"arguments[0].scrollTop += {scroll_increment};", parent_container)

    # Wait for a short moment to let the content load
    time.sleep(1)


KeyboardInterrupt: 